In [113]:
import qiskit
from qiskit.quantum_info import state_fidelity
import numpy as np
from numpy import linalg as LA
import qib
import matplotlib.pyplot as plt
import scipy

I2 = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

# Random unitary generator
def random_unitary(n):
    A = np.random.randn(n, n) + 1j * np.random.randn(n, n)
    Q, _ = np.linalg.qr(A)
    return Q

L = 4
# construct Hamiltonian
latt = qib.lattice.IntegerLattice((L,), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
#hamil = qib.IsingHamiltonian(field, 1, 0, 0).as_matrix().toarray()
hamil = qib.HeisenbergHamiltonian(field, (1,1,1), (0,0,0)).as_matrix().toarray()
perms = [[i for i in range(L)], [i for i in range(1, L)]+[0], [i for i in range(L)]]
U = scipy.linalg.expm(-1j*hamil)
U_back = scipy.linalg.expm(1j*hamil)
cU = U_back

In [102]:
# Numerical Test function for: d/G_j d/dG_i f(G)
from utils import polar_decomp, project_unitary_tangent, real_to_antisymm, antisymm_to_real, antisymm


def numerical_hessian(Glist, cU, U, L, perms, i, j, epsilon=1e-6):
    """Numerically compute d/dW1 of projected gradient dL/dV1 (Riemannian)."""
    numerical_H = []

    for _ in range(16):
        # Create unit vector in 16D real tangent space
        Z_real = np.zeros(16)
        Z_real[_] = 1.0
        Z = real_to_antisymm(Z_real.reshape(4, 4))  # 4x4 anti-Hermitian direction

        # Perturb G_j on the manifold
        #Gj_plus  = Glist[j] + epsilon * Glist[j] @ Z
        #Gj_minus = Glist[j] - epsilon * Glist[j] @ Z
        Gj_plus  = Glist[j] @ scipy.linalg.expm(+epsilon*Z)
        Gj_minus = Glist[j] @ scipy.linalg.expm(-epsilon*Z)

    
        if i==j:
            grad_plus  = ansatz_grad_vector(Glist[:j]+[Gj_plus]+Glist[j+1:], cU, U, L, perms, unprojected=True, flatten=False)[i]
            grad_minus = ansatz_grad_vector(Glist[:j]+[Gj_minus]+Glist[j+1:], cU, U, L, perms, unprojected=True, flatten=False)[i]
            dgrad = (grad_plus - grad_minus) / (2 * epsilon)  # shape (16,)
            G = dgrad.reshape(4, 4)

            # TODO: Is this projection correct?
            V = Glist[j]
            Z = V @ Z
            G = project_unitary_tangent(V, G)
            grad = ansatz_grad_vector(Glist, cU, U, L, perms, flatten=False, unprojected=True)[i]
            G -= 0.5 * (Z @ grad.conj().T @ V + V @ grad.conj().T @ Z)
            if not np.allclose(Z, project_unitary_tangent(V, Z)):
                G -= 0.5 * (Z @ V.conj().T + V @ Z.conj().T) @ grad
            G = antisymm_to_real(antisymm( V.conj().T @ G ))
        else:
            grad_plus  = ansatz_grad_vector(Glist[:j]+[Gj_plus]+Glist[j+1:], cU, U, L, perms, unprojected=False, flatten=False)[i]
            grad_minus = ansatz_grad_vector(Glist[:j]+[Gj_minus]+Glist[j+1:], cU, U, L, perms, unprojected=False, flatten=False)[i]
            dgrad = (grad_plus - grad_minus) / (2 * epsilon)  # shape (16,)
            G = dgrad.reshape(4, 4)
            
        numerical_H.append(G)
    
    return np.array(numerical_H)  # shape: (16, 4, 4)


i, j = 1,3
Glist = [random_unitary(4) for _ in range(2 * len(perms))]
H = ansatz_hessian_matrix(Glist, cU, U, L, perms, unprojected=False, flatten=False)
grad = []
for _ in range(16):
    grad.append(H[i, :, j, _].reshape(4,4))
analytical = np.array(grad)

numerical = numerical_hessian(Glist, cU, U, L, perms, i, j)

#print("Numerical 4x4 matrix:\n", numerical[0])
#print("Analytical 4x4 matrix:\n", analytical[0])
print("Difference norm:", np.linalg.norm(numerical - analytical))

In [ ]:
from optimize import optimize_circuit


Glists = []
f_iters = []
for _ in range(10):
    Glist_start = [random_unitary(4) for _ in range(2 * len(perms))]
    #Glist_start = Glist_opt_L2
    Glist, f_iter, err_iter = optimize_circuit(L, U, cU, Glist_start, perms, niter=90)
    print("Best f: ", f_iter[-1])
    print("Best err: ", err_iter[-1])
    Glists.append(Glist)
    f_iters.append(f_iter)

Best f:  -13.705436464334262
Best err:  1.0391743748902362
Best f:  -15.098297603921036
Best err:  0.7598283415468201
Best f:  -15.296720717012048
Best err:  0.7477500159701753
Best f:  -15.687599439818186
Best err:  0.45762861404699684


In [ ]:
import h5py

with h5py.File(f"../results_data/Heisenberg1d_L{L}_layers{len(perms)}_err{round(err_iter[-1], 2)}.hdf5", "w") as f:
    f.create_dataset("Glist", data=Glist)

In [133]:
L = 4
latt = qib.lattice.IntegerLattice((L,), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
hamil = qib.HeisenbergHamiltonian(field, (1,1,1), (0,0,0)).as_matrix().toarray()
U = scipy.linalg.expm(-1j*hamil)
U_back = scipy.linalg.expm(1j*hamil)
cU = U_back

print("Reusing L=4 gates for L=4: ", np.linalg.norm(ansatz(Glist_opt_perms3, U, L, perms) - cU, ord=2))

Reusing L=4 gates for L=4:  0.2474001066467117
